# KMINST Classifier using LeNet
Run all of the numbered sequentially. This notebook produces a LeNet classifier model. The cells below will produce the optimal predictions and model weights that we have submitted on Kaggle. 

##1. Functions necessary to use methods in Utils.ipynb
Please run  all of these cells. 

In [0]:
import io, os, sys, types
import nbformat

from IPython import get_ipython
from IPython.core.interactiveshell import InteractiveShell

In [0]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path
    
    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
            
class NotebookLoader(object):
    """Module Loader for IPython Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path
    
    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)
        
        print ("importing notebook from %s" % path)
                                       
        # load the notebook object
        nb = nbformat.read(path, as_version=4)
        
        
        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod
        
        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__
        
        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

class NotebookFinder(object):
    """Module finder that locates IPython Notebooks"""
    def __init__(self):
        self.loaders = {}
    
    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return
        
        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)
        
        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
      

sys.meta_path.append(NotebookFinder())

### Mount Google drive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


### Show that file exists in Google path
**Note**:  You will need to ensure that you have a folder named "**KMNIST_ENTROPY**" in your Google Drive, that contains  Utils.ipynb (and  \__init__.py)

In [4]:
ls gdrive/My\ Drive/KMNIST_ENTROPY

AlexNet.ipynb  LeNet.ipynb            PCA_Classifier.ipynb  Utils.ipynb
__init__.py    Model_Averaging.ipynb  __pycache__/


### Append the system path and import Utils.ipynb

In [0]:
sys.path.append('gdrive/My Drive/')

In [6]:
from KMNIST_ENTROPY.Utils import *

# If this cell gives a "No Module Found Error", please restart the runtime of the collab notebook 

importing notebook from gdrive/My Drive/KMNIST_ENTROPY/Utils.ipynb
Populating the interactive namespace from numpy and matplotlib
Cuda installed! Running on GPU!
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
Utils.ipynb has finished downloading


## 2. LeNet Network Architecture
Running these different networks will allow you to train a model using them. If you forget to run the networks below, you will **not** be able to use them for training. 

### Basic LeNet Network

In [0]:
class LeNet5(nn.Module):
    """
    This is a basic LeNet5 neural network.
    It has the following basic layer architecture:
    Convolution > MaxPool > Convolution > MaxPool > Fully-Connected > Fully-Connected > Fully-Connected
    
    The activation function used is ReLU
    """
    def __init__(self):
        super(LeNet5, self).__init__()
        self.c1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c3 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c5 = nn.Linear(16*5*5, 120)
        self.f6 = nn.Linear(120, 84)
        self.output = nn.Linear(84, 10)
        self.act = nn.ReLU()
    
    def forward(self, x):
        x = self.act(self.c1(x))
        x = self.act(self.s2(x))
        x = self.act(self.c3(x))
        x = self.act(self.s4(x))
        x = x.view(-1, x.size(1)*x.size(2)*x.size(3))
        x = self.act(self.c5(x))
        x = self.act(self.f6(x))
        return self.output(x)

### LeNet Network with Dropout

In [0]:
class LeNet5_Drop(nn.Module):
    """
    This is a LeNet5 neural network implemented with Dropout.
    It has the following basic layer architecture:
    Convolution > MaxPool > Convolution > MaxPool > Fully-Connected(+ Dropout) > Fully-Connected(+ Dropout) > Fully-Connected
    
    The activation function used is ReLU
    """
    def __init__(self):
        super(LeNet5_Drop, self).__init__()
        self.c1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c3 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c5 = nn.Linear(16*5*5, 120)
        self.c5dropout = nn.Dropout2d(0.5)
        self.f6 = nn.Linear(120, 84)
        self.f6dropout = nn.Dropout2d(0.5)
        self.output = nn.Linear(84, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.act(self.c1(x))
        x = self.act(self.s2(x))
        x = self.act(self.c3(x))
        x = self.act(self.s4(x))
        x = x.view(-1, x.size(1)*x.size(2)*x.size(3))
        x = self.act(self.c5(self.c5dropout(x)))
        x = self.act(self.f6(self.f6dropout(x)))
        return self.output(x)


### LeNet Network with input kernel size of 3

In [0]:
class LeNet3(nn.Module):
    """
    This is a LeNet5 neural network implemented with a input Kernel size of 3.
    It has the following basic layer architecture:
    Convolution > MaxPool > Convolution > MaxPool > Fully-Connected > Fully-Connected > Fully-Connected
    
    The activation function used is ReLU
    """
    def __init__(self):
        super(LeNet3, self).__init__()
        self.c1 = nn.Conv2d(1, 6, kernel_size=3, stride=1, padding=1) # Revised Kernel size to 3 from basic LeNet5
        self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c3 = nn.Conv2d(6, 16, kernel_size=3, stride=1) # Revised Kernel size to 3 from basic LeNet5
        self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c5 = nn.Linear(16*6*6, 120)
        self.f6 = nn.Linear(120, 84)
        self.output = nn.Linear(84, 10)
        self.act = nn.ReLU()
    
    def forward(self, x):
        x = self.act(self.c1(x))
        x = self.act(self.s2(x))
        x = self.act(self.c3(x))
        x = self.act(self.s4(x))
        x = x.view(-1, x.size(1)*x.size(2)*x.size(3))
        x = self.act(self.c5(x))
        x = self.act(self.f6(x))
        return self.output(x)

### The extended LeNet Network

In [0]:
class LeNet5_ext(nn.Module):
    """
    This is an extended LeNet5 neural network with one additional fully-connected layer. The size of each layer have been enlarged as well.
    It has the following layer architecture:
    Convolution > MaxPool > Convolution > MaxPool > Fully-Connected > Fully-Connected > Fully-Connected > Fully-Connected
    
    The activation function used is ReLU
    """
    def __init__(self):
        super(LeNet5_ext, self).__init__()
        self.c1 = nn.Conv2d(1, 18, kernel_size=5, stride=1, padding=2)
        self.s2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c3 = nn.Conv2d(18, 48, kernel_size=5, stride=1)
        self.s4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.c5 = nn.Linear(48*5*5, 600)
        self.c6 = nn.Linear(600, 300)
        self.f6 = nn.Linear(300, 100)
        self.output = nn.Linear(100, 10)
        self.act = nn.ReLU()
    def forward(self, x):
        x = self.act(self.c1(x))
        x = self.act(self.s2(x))
        x = self.act(self.c3(x))
        x = self.act(self.s4(x))
        x = x.view(-1, x.size(1)*x.size(2)*x.size(3))
        x = self.act(self.c5(x))
        x = self.act(self.c6(x))
        x = self.act(self.f6(x))
        return self.output(x)

## 3. Training with optimal hyperparameters
These are the parameters we used to train our model for the submission on Kaggle:
- Optimiser: SGD
- Learning rate: 1e-2
- Number of epochs: 10
- No Dropout
- Random Choice Data Augmentation (aug): 20 
- No weight decay

### Train model
Do **not** alter this model if you want to reporoduce the submission obtained on Kaggle. 

In [0]:
# Split
trains, valids, tr_labels, val_labels = k_split(kmnist_data, kmnist_labels, splits=1)

# Mean + std calculation
train_mean, val_mean, train_std, val_std = get_mean_std(trains, valids)

# Dataset
k_train = CustomImageTensorDataset(trains, tr_labels.long(), transform=transformed(train_mean, train_std, choice=1))
k_aug = CustomImageTensorDataset(trains, tr_labels.long(), transform=transformed(train_mean, train_std, choice=2))
k_validate = CustomImageTensorDataset(valids, val_labels.long(), transform=transformed(val_mean, val_std, choice=1))

# Choose model to use 
model = LeNet5_ext()

# Train with augmented data
model = train_model_augmented(model, k_train, k_validate, k_aug, n_epochs=10, plot=True, augs=20)

## 4. Save files 
These files can be names whatever you wish, we have just given examples. 

### Save testset predictions to csv file 
This saved file will be found in the *KMNIST_ENTROPY/results/* folder

In [0]:
save_predictions(model, kmnist_test, name="lenet_optim.csv")

### Save model for later use (if needed)
This saved file will be found in the *KMNIST_ENTROPY/models/* folder

In [0]:
save_model(model, "LeNet_optim_model.pth")